# Time resample

In [151]:
import pandas as pd
from read_json import as_json, format_time
import numpy as np
import matplotlib.pyplot as plt

In [98]:
data = pd.read_csv("data_sample.csv")
data["@timestamp"] = pd.to_datetime(data["@timestamp"].map(format_time))  # convert to pandas datetime
id_nan =data.system.isna()
data_no_nan = data.loc[~id_nan]

In [155]:
df_qaserver_resampled

,cores,user_pct,idle_pct,system_pct,actual_used_bytes,actual_used_pct,swap_free,swap_total,free,swap,bytes_total,bytes_total_norm,swap_norm,out_bytes,out_bytes_norm,in_bytes,in_bytes_norm
@timestamp,,,,,,,,,,,,,,,,,
2023-03-01 00:00:00,4,0.016037,0.965000,0.018963,5.764923e+09,0.2237,2.359289e+10,2.952686e+10,2.000384e+10,0.200968,2.576876e+10,0.392488,0.251732,0.000000e+00,0.000000,0.000000e+00,0.000000
2023-03-01 00:10:00,4,NaN,NaN,NaN,5.756232e+09,0.2234,2.359266e+10,2.952686e+10,2.001253e+10,0.200976,2.576876e+10,0.392488,0.251742,8.687552e+08,0.001072,6.638383e+08,0.004690
2023-03-01 00:20:00,4,0.003900,0.985150,0.010950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.739137e+09,0.002147,1.328780e+09,0.009387
2023-03-01 00:30:00,4,NaN,NaN,NaN,5.759775e+09,0.2235,2.359901e+10,2.952686e+10,2.000899e+10,0.200761,2.576876e+10,0.392488,0.251472,5.800311e+08,0.000716,4.432566e+08,0.003131
2023-03-01 00:40:00,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.713257e+08,0.001075,6.658129e+08,0.004703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-31 23:10:00,4,0.001575,0.992950,0.005475,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000,0.000000e+00,0.000000
2023-03-31 23:20:00,4,0.001175,0.994525,0.004300,5.752390e+09,0.2232,2.380221e+10,2.952686e+10,2.001637e+10,0.193879,2.576876e+10,0.392488,0.242852,6.789458e+08,0.000838,3.906981e+08,0.002760
2023-03-31 23:30:00,4,NaN,NaN,NaN,5.756232e+09,0.2234,2.380553e+10,2.952686e+10,2.001253e+10,0.193767,2.576876e+10,0.392488,0.242711,NaN,NaN,NaN,NaN


In [166]:
cpu_df = pd.read_csv("cpu_df.csv",index_col=0)
mem_df = pd.read_csv("memory_df.csv", index_col=0)
net_df = pd.read_csv("network_df.csv", index_col=0)

frames = [cpu_df, mem_df, net_df]
df = pd.concat(frames)  # si jala
df.sort_index(inplace=True)
#df["@timestamp"] = data_no_nan["@timestamp"]
df.insert(0, "@timestamp", data_no_nan["@timestamp"])
df_time = df

# do it by host -->  then relate by host
df_qaserver = df_time.loc[df_time["host"] == "QASERVER"].drop("host", axis=1)
# df_time["@timestamp"].resample("5Min")
df_qaserver_resampled = df_qaserver.resample('10Min', on="@timestamp").mean()
df_qaserver_resampled["cores"] = 4  # the server has the same number of cores
df_qaserver_resampled.to_csv("resampled/qaserver_resampled.csv")
#plt.plot(df_qaserver_resampled.index, df_qaserver_)